In [3]:
!pip install PyMuPDF
!pip install pytesseract
!pip install pdf2image
!pip install nltk


!apt-get install -y poppler-utils
!apt-get install -y tesseract-ocr


import fitz
import pytesseract
from pdf2image import convert_from_path
import nltk
import re


nltk.download('punkt')

# Function to convert PDF to text
def pdf_to_text(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    return text

# Function to extract images from PDF
def pdf_to_images_ocr(pdf_path):
    images = convert_from_path(pdf_path)
    ocr_text = ""
    for image in images:
        ocr_text += pytesseract.image_to_string(image)
    return ocr_text


def preprocess_text(text):

    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

    sentences = nltk.sent_tokenize(text)

    tokenized_text = [nltk.word_tokenize(sentence) for sentence in sentences]
    return tokenized_text


pdf_path = '/content/sample1.pdf'
text_from_pdf = pdf_to_text(pdf_path)
text_from_images = pdf_to_images_ocr(pdf_path)
combined_text = text_from_pdf + text_from_images
preprocessed_text = preprocess_text(combined_text)

print(preprocessed_text)


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.4).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[['Ihr', 'Logo', 'Musterfirma', 'MusterstraRe', '23', '12345', 'Musterhausen', 'Gutschriftsempfanger', 'StraBe', 'Hausnummer', 'PLZ', 'Ort', 'Musterfirma', 'AG', 'Musterstrake', '23', '12345', 'Musterhausen', 'Datum', '01032019', 'Gutschrift', 'Nr', '20191004', 'Ihre', 'UmSt', 'ID', 'DE123456789', 'Gutschriftssdatum', 'entspricht', 'LieferLeistungsdatum', 'Gutschrift', 'Sehr', 'geehrter', 'Herr', 'Schmidt', 'gemaf', 'unserer', 'Vereinbarung', 'schreiben', 'wir', 'Ihnen', 'folgende', 'Leistungen', 'gut', 'Position', 'Anzahl', 'Einheit', 'Bezeichnung', 'Einzelpreis', 'Gesamtpreis', '1', '5', 'Sttick', 'Musterleistung', '300', '1500', '2', '3', 'Stuck', 'Musterleistung', '500', '1500', 'Nettopreis', '3000', 'Zzgl', '19', 'USt', '570', 'Gutschriftbetrag', '3570', 'Wir', 'Uberweisen', 'Ihnen', 'den', 'Gutschriftbetrag', 'in', 'den', 'nachsten', 'Tagen', 'auf', 'Ihr', 'Konto', 'Mit', 'freundlichen', 'Gruen', 'Max', 'Mustermann', 'Musterfirma', 'GmbH', 'Kreditinstitut', 'Commerzbank', 'UStID'

In [4]:
def process_pdf(pdf_path):
    text_from_pdf = pdf_to_text(pdf_path)
    text_from_images = pdf_to_images_ocr(pdf_path)
    combined_text = text_from_pdf + text_from_images
    preprocessed_text = preprocess_text(combined_text)
    return preprocessed_text

pdf_path = '/content/sample1.pdf'
final_processed_text = process_pdf(pdf_path)

for sentence in final_processed_text:
    print(' '.join(sentence))

Ihr Logo Musterfirma MusterstraRe 23 12345 Musterhausen Gutschriftsempfanger StraBe Hausnummer PLZ Ort Musterfirma AG Musterstrake 23 12345 Musterhausen Datum 01032019 Gutschrift Nr 20191004 Ihre UmSt ID DE123456789 Gutschriftssdatum entspricht LieferLeistungsdatum Gutschrift Sehr geehrter Herr Schmidt gemaf unserer Vereinbarung schreiben wir Ihnen folgende Leistungen gut Position Anzahl Einheit Bezeichnung Einzelpreis Gesamtpreis 1 5 Sttick Musterleistung 300 1500 2 3 Stuck Musterleistung 500 1500 Nettopreis 3000 Zzgl 19 USt 570 Gutschriftbetrag 3570 Wir Uberweisen Ihnen den Gutschriftbetrag in den nachsten Tagen auf Ihr Konto Mit freundlichen Gruen Max Mustermann Musterfirma GmbH Kreditinstitut Commerzbank UStID DE24324567 MusterstraRe 23 IBAN DE3423 4562 3435 6765 HRB 1234567B 12345 Musterhausen BIC COBADEFFXXX Amtsgericht Charlottenburg Tel 40 030 12345678 Kto Inh Max Mustermann Geschaftsfilhrer Max Mustermann EMail infomusterde Webseite wwwfirmade


In [5]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")


def extract_information(text):
    nlp = pipeline("ner", model=model, tokenizer=tokenizer)
    return nlp(text)

def classify_document(text):
    classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)
    return classifier(text)

# Function to translate text
def translate_text(text, target_language="es"):
    # T5 is versatile and can handle tasks like translation with task-specific prefixes
    input_text = f"translate English to {target_language}: {text}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs["input_ids"], max_length=512, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [6]:
!pip install streamlit --quiet
!pip install --upgrade streamlit
!pip install PyMuPDF pdf2image pytesseract transformers nltk --quiet
!pip install pyngrok --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.3 MB/s eta 0:00:00


In [7]:
%%writefile app.py
import streamlit as st
import fitz  # PyMuPDF
from pdf2image import convert_from_path
import pytesseract
import nltk
import re
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

# Functions for processing
def pdf_to_text(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    return text

def pdf_to_images_ocr(pdf_path):
    images = convert_from_path(pdf_path)
    ocr_text = ""
    for image in images:
        ocr_text += pytesseract.image_to_string(image)
    return ocr_text

def preprocess_text(text):
    nltk.download('punkt', quiet=True)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    sentences = nltk.sent_tokenize(text)
    tokenized_text = [nltk.word_tokenize(sentence) for sentence in sentences]
    return tokenized_text

tokenizer = AutoTokenizer.from_pretrained("t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")

def extract_information(text):
    nlp = pipeline("ner", model=model, tokenizer=tokenizer)
    return nlp(text)

def classify_document(text):
    classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)
    return classifier(text)

def translate_text(text, target_language="es"):
    input_text = f"translate English to {target_language}: {text}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs["input_ids"], max_length=512, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def process_pdf(pdf_path):
    text_from_pdf = pdf_to_text(pdf_path)
    text_from_images = pdf_to_images_ocr(pdf_path)
    combined_text = text_from_pdf + text_from_images
    preprocessed_text = preprocess_text(combined_text)
    return preprocessed_text, combined_text

def main():
    st.set_page_config(page_title="AI Document Processing", layout="wide", initial_sidebar_state="expanded")
    st.title("📝 AI-Powered Document Processing Pipeline")
    st.write("Upload a document to process.")

    uploaded_file = st.file_uploader("Choose a PDF file", type="pdf")

    if uploaded_file is not None:
        with open("uploaded_file.pdf", "wb") as f:
            f.write(uploaded_file.getbuffer())

        st.write("Processing the document...")
        preprocessed_text, combined_text = process_pdf("uploaded_file.pdf")

        st.subheader("Extracted Text:")
        st.text_area("Combined Text", combined_text, height=300)

        col1, col2, col3 = st.columns(3)

        with col1:
            if st.button("Extract Information"):
                st.write("Extracting information...")
                extracted_info = extract_information(combined_text)
                st.write("Extracted Information:")
                st.json(extracted_info)

        with col2:
            if st.button("Classify Document"):
                st.write("Classifying document...")
                classification = classify_document(combined_text)
                st.write("Document Classification:")
                st.json(classification)

        with col3:
            if st.button("Translate Text"):
                st.write("Translating text...")
                translated_text = translate_text(combined_text)
                st.write("Translated Text:")
                st.text_area("Spanish Translation", translated_text, height=200)

if __name__ == "__main__":
    main()


Writing app.py


In [8]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.122.182.187:8501

npx: installed 22 in 7.852s
your url is: https://all-buses-report.loca.lt
2024-06-11 17:53:09.364177: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-11 17:53:09.364263: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-11 17:53:09.368348: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-11 17:53:10.985845: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find